In [1]:
import numpy as np

from bvh_skeleton import nturgbd_skeleton

In [2]:
input_dir = '../data/ntu/xsub/val_data.npy'
output_dir = '../data/ntu/xsub/val_data.bvh'

In [3]:
data_o = np.load(input_dir)
data_o.shape

(16487, 3, 300, 25, 2)

In [4]:
N, C, T, V, M = data_o.shape

In [5]:
data_o = data_o[0, :, :, :, 0]
data_o.shape

(3, 300, 25)

In [6]:
data = data_o.transpose((1, 2, 0))
data.shape

(300, 25, 3)

In [7]:
np.set_printoptions(suppress=True)
for d in data:
    print(d)

[[ 0.337624    0.02674085  2.30821   ]
 [ 0.3622169   0.3414172   2.253364  ]
 [ 0.3840512   0.6447685   2.186779  ]
 [ 0.3636071   0.7669529   2.202303  ]
 [ 0.2755833   0.5181808   2.317265  ]
 [ 0.3051753   0.3934617   2.053786  ]
 [ 0.3308939   0.2092391   2.24904   ]
 [ 0.3400474   0.1486648   2.318024  ]
 [ 0.4358261   0.4854883   2.095815  ]
 [ 0.4191501   0.2030537   2.054853  ]
 [ 0.3731624   0.00245875  2.084983  ]
 [ 0.3678356  -0.06406707  2.129233  ]
 [ 0.2800644   0.03306213  2.332707  ]
 [ 0.3150388  -0.2696033   2.514304  ]
 [ 0.4371791  -0.7081608   2.576044  ]
 [ 0.4137459  -0.786814    2.586746  ]
 [ 0.3844772   0.01965216  2.210543  ]
 [ 0.409483   -0.3429189   2.279757  ]
 [ 0.4890379  -0.7281572   2.405067  ]
 [ 0.3920269  -0.7759282   2.321116  ]
 [ 0.3790364   0.5705596   2.205542  ]
 [ 0.3440572   0.08721786  2.357158  ]
 [ 0.3360894   0.1537393   2.2889    ]
 [ 0.3374537  -0.1451203   2.147256  ]
 [ 0.320399   -0.00727248  2.16289   ]]
[[ 0.3353383   0.0285979

## Convert 3D pose to BVH

In [8]:
# numpy array with shape (300, 25, 3)
pose3d_world = data

skeleton = nturgbd_skeleton.NTURGBDSkeleton()
channels, header = skeleton.poses2bvh(pose3d_world, output_file=output_dir)

C:\Users\80596\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\80596\Anaconda3\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [9]:
channels = np.asarray(channels)
channels.shape

(300, 78)

In [10]:
channels[:, 3:].reshape(T, V, C).shape

(300, 25, 3)

In [11]:
for i in header.nodes:
    print(i, header.nodes[i].offset)

SpineBase [0. 0. 0.]
SpineMiddle [0.         0.         0.32294336]
Neck [0.         0.         0.07828289]
Head [0.         0.         0.12525029]
LeftShoulder [0.15947597 0.         0.        ]
LeftElbow [0.29193532 0.         0.        ]
LeftWrist [0.24801289 0.         0.        ]
LeftHand [0.07172197 0.         0.        ]
RightShoulder [-0.15375338  0.          0.        ]
RightElbow [-0.28490108  0.          0.        ]
RightWrist [-0.22131583  0.          0.        ]
RightHand [-0.0787827  0.         0.       ]
LeftHip [0.06404494 0.         0.        ]
LeftKnee [ 0.          0.         -0.28517854]
LeftAnkle [ 0.          0.         -0.42298725]
LeftFoot [ 0.         -0.12186217  0.        ]
RightHip [-0.10173655  0.          0.        ]
RightKnee [ 0.          0.         -0.35811609]
RightAnkle [ 0.          0.         -0.40760019]
RightFoot [ 0.         -0.12734154  0.        ]
Spine [0.         0.         0.23799989]
LeftHandTip [0.06412542 0.         0.        ]
LeftThumb 

## Test

Get v.bvh Manually.

Result: val_data.bvh -> v.bvh

				End Site 
				{
					OFFSET	 0.00	 0.00	 0.001
				}

bvh-converter v.bvh

Result: v.bvh -> v_worldpos.csv

In [12]:
import csv

with open('../data/ntu/xsub/v_worldpos.csv') as f:
    f_csv = list(csv.reader(f))
    headers = np.array(f_csv[0])
    rows = np.array(f_csv[1:])

In [13]:
data_test = []
for key in skeleton.keypoint2index:
    for i, header in enumerate(headers):
        if key + '.X' == header:
            data_test.append(rows[:, i:i + 3])

In [14]:
data_test = np.asarray(data_test, dtype='float32')
data_test = data_test.transpose(1, 0, 2)
data_test.shape

(300, 25, 3)

In [15]:
data.shape

(300, 25, 3)

In [16]:
dists = []
for i in range(data.shape[0]):
    dist = np.linalg.norm(data[i] - data_test[i], axis=1)
    if not np.all(data[i] == 0):
        dists.append(dist)
dists = np.asarray(dists)

In [17]:
np.mean(dists)

0.064083666